In [1]:
from typing import TypedDict, Annotated
import operator

from langgraph.graph import StateGraph, END

from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_community.tools.tavily_search import TavilySearchResults

from dotenv import load_dotenv
load_dotenv()

/Users/rachneet/projects/agents_experimental/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen3-Next-80B-A3B-Instruct",
    max_new_tokens=1024,
    temperature=0,
)

chat_model = ChatHuggingFace(llm=llm)

In [3]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [4]:
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver

# memory = SqliteSaver.from_conn_string(":memory:")
conn = sqlite3.connect("agent_memory.db", check_same_thread=False)
memory = SqliteSaver(conn)

In [5]:
class Agent:

    def __init__(self, model, tools, checkpointer, system=""):

        self.system = system
        graph = StateGraph(AgentState)

        graph.add_node("llm", self.call_model)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {
                True: "action", False: END
            }
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")

        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_model(self, state: AgentState):
        messages = state["messages"]
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {"messages": [message]}

    def take_action(self, state: AgentState):
        tool_calls = state["messages"][-1].tool_calls
        results = []

        for t in tool_calls:
            print(f"Calling: '{t}' ")
            result = self.tools[t["name"]].invoke(t["args"])
            results.append(ToolMessage(
                tool_call_id=t["id"], 
                name=t["name"], 
                content=str(result)
            ))
        
        return {"messages": results}

    def exists_action(self, state: AgentState):
        result = state["messages"][-1]
        return len(result.tool_calls) > 0

In [10]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

tool = TavilySearchResults(max_results=2)

/var/folders/nm/srw3ygl55jbgpddqwgysqjpm0000gn/T/ipykernel_31337/1493036220.py:7: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tool = TavilySearchResults(max_results=2)


In [ ]:
abot = Agent(chat_model, [tool], checkpointer=memory, system=prompt)

In [31]:
messages = [HumanMessage(content="What is the weather in Darmstadt, Germany?")]

In [32]:
thread = {
    "configurable": {"thread_id": "1"}
}

In [33]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query": "current weather in Darmstadt, Germany"}', 'name': 'tavily_search_results_json', 'description': None}, 'id': 'call_1260f4beadc84ebcac4b0a', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 261, 'total_tokens': 291}, 'model_name': 'Qwen/Qwen3-Next-80B-A3B-Instruct', 'system_fingerprint': '', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--391e8e5b-1684-4a37-a1ef-998af1c65618-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Darmstadt, Germany'}, 'id': 'call_1260f4beadc84ebcac4b0a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 261, 'output_tokens': 30, 'total_tokens': 291})]
Calling: '{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Darmstadt, Germany'}, 'id': 'call_1260f4beadc84ebcac4b0a', 'type': 'tool_call'}' 
[ToolMessage(content='[{\'title\': \'Weat

In [34]:
messages = [HumanMessage(content="What is the weather in Munich, Germany?")]
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query": "current weather in Munich, Germany"}', 'name': 'tavily_search_results_json', 'description': None}, 'id': 'call_bf813d349e2549659ae22a', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 1437, 'total_tokens': 1465}, 'model_name': 'Qwen/Qwen3-Next-80B-A3B-Instruct', 'system_fingerprint': '', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ed49adc9-af52-474b-9b91-bcff33c5a688-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Munich, Germany'}, 'id': 'call_bf813d349e2549659ae22a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1437, 'output_tokens': 28, 'total_tokens': 1465})]
Calling: '{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Munich, Germany'}, 'id': 'call_bf813d349e2549659ae22a', 'type': 'tool_call'}' 
[ToolMessage(content="[{'title': 'Munich weath

In [35]:
messages = [HumanMessage(content="What is warmer?")]
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='Between Darmstadt and Munich, **Darmstadt** is warmer based on the current weather data for October 22, 2025:\n\n- **Darmstadt**:  \n  - Temperature: ~60°F (15.5°C) high / 50°F (10°C) low  \n  - Feels like: 61°F (16°C)\n\n- **Munich**:  \n  - Temperature: ~54°F (12°C) high / 50°F (10°C) low (with light rain)  \n\nDarmstadt is experiencing slightly higher temperatures, particularly in its daytime highs, making it warmer than Munich at this time.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 144, 'prompt_tokens': 2628, 'total_tokens': 2772}, 'model_name': 'Qwen/Qwen3-Next-80B-A3B-Instruct', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='run--0fef6452-9b1a-4096-bd2b-8ad497940e2e-0', usage_metadata={'input_tokens': 2628, 'output_tokens': 144, 'total_tokens': 2772})]


In [36]:
messages = [HumanMessage(content="What is warmer?")]
# change thread id to see the history vanish
thread = {
    "configurable": {"thread_id": "2"}
}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content="Could you clarify what you're comparing? For example, are you asking about temperatures in different places, materials, or conditions? Let me know so I can help!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 254, 'total_tokens': 287}, 'model_name': 'Qwen/Qwen3-Next-80B-A3B-Instruct', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='run--ab2b4a58-22f8-47c4-892a-3a3163a57b95-0', usage_metadata={'input_tokens': 254, 'output_tokens': 33, 'total_tokens': 287})]


## Streaming Tokens

In [13]:
import aiosqlite
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver

async_conn = aiosqlite.connect("async_agent_memory.db", check_same_thread=False)
memory = AsyncSqliteSaver(async_conn)

abot = Agent(chat_model, [tool], system=prompt, checkpointer=memory)

In [15]:
messages = [HumanMessage(content="What is the weather in Darmstadt, Germany?")]
thread = {"configurable": {"thread_id": "4"}}

async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")

Calling: '{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Darmstadt, Germany today'}, 'id': 'call_92e4f2badded43fb9a5c29', 'type': 'tool_call'}' 
As| of today|, October| 22,| 202|5, the weather| in Darmstadt|, Germany, is| **overcast** with| a high of |63°F (|17°C).| 

Tonight, conditions| will remain cloudy, with| a couple of late|-night showers expected. The low| temperature will be around 5|3°F (1|2°C). 

|Rain is expected to| start within the next minute|, according to the| minute-by-minute forecast.| The upcoming days will| continue to feature show|ery and windy conditions through| Thursday evening.

For real|-time updates, it|'s always best to| check a live weather| service like Accu|Weather or a similar platform|.|